# Assignment 3


# ETL +Text Cleaning+Text Extraction

In [9]:
!pip install -q tokenizer transformers scikit-learn pypdf==3.16.0 nltk pandarallel pandas==2.1.0 datasets torch --user

In [ ]:
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import datasets
import transformers

In [12]:
resume_data=pd.read_csv("C:/Users/aditya1/Desktop/Assignment-3/Resume/Resume.csv")
resume_data=resume_data.drop(["Resume_html"],axis=1)

In [14]:
resume_data.head(2)

,ID,Resume_str,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...",HR


In [15]:
#getting all text from a pdf
from pypdf import PdfReader

def pdf_text(filePath:str)->str:
    reader = PdfReader(filePath)
    text=""
    for page in reader.pages:
        text+=page.extract_text()
    return text


In [ ]:
import nltk
nltk.download('all')

In [19]:
#doing text cleaninig and tokenization
from nltk import pos_tag
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import string
import re

puncuation=set(string.punctuation)
stop_words_english=set(stopwords.words("english"))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    sentences = sent_tokenize(text)
    features = {'feature': ""}

    for sent in sentences:
        for criteria in ['skills', 'education']:
            if criteria in sent:
                words = word_tokenize(sent)
                words = [word for word in words if word not in stop_words_english]
                # POS tagger to identify and remove stop words and other irrelevant words
                tagged_words = pos_tag(words)
                filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                features['feature'] += " ".join(filtered_words)

    return features

In [20]:
progress_bar=tqdm(range(len(resume_data)))
# extract text from csv and get the text from pdf
def process(df):
    #uncomment when using pdf_text function .. done to save time
    id=df['ID']
    category=df['Category'] 
    text=pdf_text(f"C:/Users/aditya1/Desktop/Assignment-3/data/data/{category}/{id}.pdf")
   # text=str(df["Resume_str"])
    features=preprocess_text(text)
    df['Feature']=features['feature']
    progress_bar.update(1)
    return df
      
#applying processing to resume_data
resume_data=resume_data.apply(process,axis=1)
resume_data=resume_data.drop(columns=['Resume_str'])

  0%|          | 0/2484 [00:00<?, ?it/s]

In [ ]:
resume_data.to_csv("resume_data.csv",index=False)

In [105]:
resume_data=pd.read_csv("resume_data.csv")

In [106]:
resume_data.isnull().sum()

ID          0
Category    0
Feature     1
dtype: int64

In [107]:
resume_data.dropna(inplace=True,axis=0)

In [108]:
resume_data.head()

,ID,Category,Feature
0,16852973,HR,hr administrator marketing associate hr admini...
1,22323967,HR,hr specialist hr operations summary media prof...
2,33176873,HR,hr director summary years experience recruitin...
3,27018550,HR,hr specialist summary dedicated driven dynamic...
4,17812897,HR,hr manager skill highlights hr skills hr depar...


In [110]:
resume_data['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [112]:
label_mapping = {
    'HR': 0,
    'DESIGNER': 1,
    'INFORMATION-TECHNOLOGY': 2,
    'TEACHER': 3,
    'ADVOCATE': 4,
    'BUSINESS-DEVELOPMENT': 5,
    'HEALTHCARE': 6,
    'FITNESS': 7,
    'AGRICULTURE': 8,
    'BPO': 9,
    'SALES': 10,
    'CONSULTANT': 11,
    'DIGITAL-MEDIA': 12,
    'AUTOMOBILE': 13,
    'CHEF': 14,
    'FINANCE': 15,
    'APPAREL': 16,
    'ENGINEERING': 17,
    'ACCOUNTANT': 18,
    'CONSTRUCTION': 19,
    'PUBLIC-RELATIONS': 20,
    'BANKING': 21,
    'ARTS': 22,
    'AVIATION': 23
}

# Replace labels with numerical values
resume_data['Category'] = resume_data['Category'].map(label_mapping)


In [113]:
resume_data.isnull().sum()

ID          0
Category    0
Feature     0
dtype: int64

In [114]:
resume_data=resume_data[['Category','Feature']]

In [115]:
resume_data.head()

,Category,Feature
0,0,hr administrator marketing associate hr admini...
1,0,hr specialist hr operations summary media prof...
2,0,hr director summary years experience recruitin...
3,0,hr specialist summary dedicated driven dynamic...
4,0,hr manager skill highlights hr skills hr depar...


# Converting the dataset in a format supported by hugging face and pussing the same to hugging face

In [116]:
from datasets import Dataset


dataset = Dataset.from_pandas(resume_data)

C:\ProgramData\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [117]:
dataset

Dataset({
    features: ['Category', 'Feature', '__index_level_0__'],
    num_rows: 2483
})

In [ ]:
dataset.push_to_hub("Resume_parser")

# Get data from hugging face hub

In [ ]:
from datasets import load_dataset

dataset = load_dataset("AdityaAI9/Resume_parser")


# Fine Tunning a model

In [120]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Category', 'Feature', '__index_level_0__'],
        num_rows: 2483
    })
})

In [121]:
dataset = dataset.remove_columns('__index_level_0__')

In [122]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Category', 'Feature'],
        num_rows: 2483
    })
})

In [123]:
from datasets import DatasetDict, Dataset

# Load dataset
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({
        "label": [label for label in dataset['train']['Category']],
        "text": [text  for text in dataset['train']['Feature']]
    })
})

# Check the dataset
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2483
    })
})


In [124]:
dataset_dict['train']

Dataset({
    features: ['label', 'text'],
    num_rows: 2483
})

In [125]:
dataset_dict['train'][0]

{'label': 0,
 'text': 'hr administrator marketing associate hr administrator summary dedicated customer service manager years experience hospitality customer service management respected builder leader customer focused teams strives instill shared enthusiastic commitment customer service highlights focused customer satisfaction team management marketing savvy conflict resolution techniques training development skilled multi tasker client relations specialist accomplishments missouri dot supervisor training certification certified ihg customer loyalty marketing segment hilton general manager training certification accomplished trainer cross server hospitality systems hilton onq micros opera pms fidelio opera reservation system ors holidex completed courses seminars customer service sales strategies inventory control loss prevention safety time management leadership performance assessment experience hr administrator marketing associate hr administrator dec current company name city state

In [126]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [127]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [128]:
tokenized_imdb = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2483 [00:00<?, ? examples/s]

In [129]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [130]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=24)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,  # Increased learning rate
    per_device_train_batch_size=32,  # Increased batch size
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Decreased number of epochs
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
